<a href="https://colab.research.google.com/github/data4class/handwrittendigits/blob/main/Portfolio_Optimisation_I_(2025).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import time

# Define stock symbols
stocks = ['RELIANCE.NS', 'INFY.NS', 'ADANIPORTS.NS', 'POWERGRID.NS', 'SBIN.NS']

# Download data for each stock individually
all_prices = []
all_returns = []

for stock in stocks:
        ticker = yf.Ticker(stock)
        hist = ticker.history(period='5y')

        # Extract close prices
        prices = hist['Close'].dropna()

        # Calculate daily returns
        returns = prices.pct_change().dropna()

        all_prices.append(prices.tolist())
        all_returns.append(returns.tolist())

        print(f"Downloaded {stock}: {len(prices)} days, {len(returns)} returns")

        # Small delay to be respectful to the API
        time.sleep(0.1)

# Create DataFrame for easier analysis
returns_df = pd.DataFrame()
prices_df = pd.DataFrame()
for i, stock in enumerate(stocks):
    if i < len(all_returns):
        # Align all return and price series by date
        returns_df[stock] = pd.Series(all_returns[i])
        prices_df[stock] = pd.Series(all_prices[i])


In [ ]:
w1 = [1/5, 1/5, 1/5, 1/5, 1/5]
w2 = [.25, .25, .10, .20, .20]

In [ ]:
def covariance(X, Y):
    # return covariance between X and Y
    if(len(X) != len(Y)):
        return("Not compatible")
    n = len(X)
    mx = sum(X)/n
    my = sum(Y)/n
    s = 0
    for (x, y) in zip(X, Y):
        s += (x - mx)*(y - my)
    return(s/n)

def covariance_matrix(D):
    n = len(D)
    k = list(D.keys())
    C = [[] for i in range(n)]
    for i in range(n):
        for j in range(n):
            # Compute covariance between D[k[i]] and D[k[j]]
            C[i].append(covariance(D[k[i]], D[k[j]]))
    return(C)


In [ ]:
import numpy as np

def pRisk(w, C):
    return(np.sqrt(w.T*C*w).item())

def pReturn(w, r):
    return(w.T*r)

def dpr(w, R):
    return(w.T*R)

##Sharpe Ratio

def sharpe(rt, rsk, rf = 0.06):
    return((rt - rf)/rsk)


In [ ]:
C = returns_df.cov()
daily_return = returns_df.sum()/5
C = np.matrix(C)
w1 = np.matrix(w1).reshape(-1, 1)
w2 = np.matrix(w2).reshape(-1, 1)
R = np.matrix(list(daily_return)).reshape(-1, 1)

risk = pRisk(w1, C)
print(risk)
risk2 = pRisk(w2, C)
print(risk2)


In [ ]:
annual_return = [sum(returns_df[symbol])/5 for symbol in stocks]


## Compare Sharpe ratio
rsk1 = pRisk(w1, C)
rt1  = (annual_return*w1).item()

rsk2 = pRisk(w2, C)
rt2  = (annual_return*w2).item()

s1 = sharpe(rt1, rsk1)
s2 = sharpe(rt2, rsk2)


In [ ]:
print(s1, s2)
print(rsk1, rsk2)
print(rt1, rt2)

In [ ]:
import random
def generate_random_weights(n, N = 10000):
    Wgt =[]#  [n*[0] for i in range(N)]
    w = n*[3]
    for i in range(N):
        while(not all([x >-3 and x<3 for x in w])):
            w = [random.uniform(-1, 1) for _ in  range(n)]
            w = [wi/sum(w) for wi in w]
        Wgt.append(w)
        w = n*[3]
    return(Wgt)

n = len(stocks)
Wg = np.matrix(generate_random_weights(n, N=25000))

rsk = [pRisk(w.T, C) for w in Wg]
rt  = [(annual_return*w.T).item() for w in Wg]


In [ ]:
import matplotlib.pyplot as plt
plt.scatter(rsk,rt,color = 'orange', marker = 'o', linewidth = .3)

###Minimum Risk and maximum Sharpe ratio point
min_risk_idx = np.argmin(rsk)

x_min = rsk[min_risk_idx]
y_min = rt[min_risk_idx]

max_sr_idx = np.argmax([sharpe(rt1,rsk1) for (rt1,rsk1) in zip(rt,rsk)])

x_msr = rsk[max_sr_idx]
y_msr = rt[max_sr_idx]

plt.scatter(x_min,y_min,color='blue', marker = '*', linewidth = 2)
plt.scatter(x_msr,y_msr,color='green', marker = 's', linewidth = 2)


###Individual securities portfolio
rsk_individual = np.sqrt(C.diagonal())
rt_individual = annual_return
plt.scatter(list(rsk_individual), list(rt_individual), color='black', marker = 'x', linewidth = 1)
plt.show()